In [7]:
import gensim.utils as utils
from tqdm import tqdm
import torch
import torch.nn as nn

In [8]:
corpus_file = open("./data/train_shuf.txt")

n_lines = 100000

corpus = []

for i in tqdm(range(n_lines)):
    corpus.append(utils.simple_preprocess(corpus_file.readline(), min_len=1))


100%|██████████| 100000/100000 [00:01<00:00, 56624.06it/s]


In [9]:
words = list(set([word for line in corpus for word in line]))
words.sort()
words = [''] + words

In [10]:
print(words[:10])

['', 'a', 'aa', 'aaa', 'aaaa', 'aachen', 'aaktualne', 'aalen', 'aalto', 'aami']


In [11]:
word_to_num = {w: i for i, w in enumerate(words)}
num_to_word = {i: w for i, w in enumerate(words)}

dict_size = len(words)

def word_to_tensor(word):
    res = torch.zeros(dict_size)
    res[word_to_num[word]] = 1
    return res

def sentence_to_tensor(sentence):
    res = torch.zeros(len(sentence), dict_size)
    for i in range(len(sentence)):
        res[i,:] = word_to_tensor(sentence[i])

    return res

def index_to_tensor(index):
    res = torch.zeros(dict_size)
    res[index] = 1
    return res

def vec_to_tensor(vec):
    res = torch.zeros(len(vec), dict_size)
    for i in range(len(vec)):
        res[i,:] = index_to_tensor(vec[i])

    return res

In [90]:
s = ['', 'a', 'aa', words[-1]]
print(sentence_to_tensor(s))

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])


In [13]:
input_corpus = corpus[:]
output_corpus = [sentence[1:] + [''] for sentence in corpus]
print(input_corpus[:2])
print(output_corpus[:2])

[['system', 'euroraty', 'chcesz', 'kupować', 'więcej', 'niż', 'gdzie', 'indziej'], ['parlament', 'zdecydował', 'jednak', 'inaczej', 'i', 'przyjął', 'w', 'ustawie', 'z', 'dnia', 'r', 'postępowanie', 'orzeczniczo', 'lekarskie']]
[['euroraty', 'chcesz', 'kupować', 'więcej', 'niż', 'gdzie', 'indziej', ''], ['zdecydował', 'jednak', 'inaczej', 'i', 'przyjął', 'w', 'ustawie', 'z', 'dnia', 'r', 'postępowanie', 'orzeczniczo', 'lekarskie', '']]


In [14]:
for i in range(len(corpus)):
    input_corpus[i] = [word_to_num[word] for word in input_corpus[i]]
    output_corpus[i] = [word_to_num[word] for word in output_corpus[i]]

print(input_corpus[:2])
print(output_corpus[:2])

[[122457, 28334, 13707, 52949, 139499, 72059, 32307, 39789], [82818, 154432, 42384, 39589, 38855, 100323, 135873, 133685, 148224, 20561, 103244, 92055, 80063, 54797]]
[[28334, 13707, 52949, 139499, 72059, 32307, 39789, 0], [154432, 42384, 39589, 38855, 100323, 135873, 133685, 148224, 20561, 103244, 92055, 80063, 54797, 0]]


In [81]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [82]:
class RNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(RNN, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

In [ ]:
class DataLoader:
    def __init__(self, input_corpus, output_corpus, batch_size):
        self.input_corpus = input_corpus
        self.output_corpus = output_corpus
        self.batch_size = batch_size
        self.corpus_size = len(input_corpus)

    def __iter__(self):
        #TODO - fix to return one_hot
        for i in range(0, self.corpus_size, self.batch_size):
            yield self.input_corpus[i: i+batch_size],
                self.output_corpus[i: i+batch_size]


In [91]:
# Instantiate the model with hyperparameters
model = RNN(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

# Define hyperparameters
n_epochs = 100
lr=0.01
batch_size = 

loader = DataLoader(input_corpus = input_corpus, output_corpus=output_corpus, batch_size=)


# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.8)


In [ ]:
#TODO unfinished

# Training Run
for epoch in range(1, n_epochs + 1):
    for batch_x, batch_y in loader:

        optimizer.zero_grad() 

        # output, hidden = model(input_seq)
        # output = output.to(device)
        # target_seq = target_seq.to(device)

        # loss = criterion(output, target_seq.view(-1).long())


        loss.backward()
        optimizer.step() 
        
        if epoch%10 == 0:
            print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
            print("Loss: {:.4f}".format(loss.item()))